In [1]:
import os
import sys
from common import commons
home = commons.home
extra_storage = commons.extra_storage
from features_preprocess import BED_binning
from features_preprocess import BED_Preprocess, CADD_Preprocess,DANN_Preprocess,Eigen_Preprocess,GenoCanyon_Preprocess
import subprocess
import pandas as pd
from features_preprocess import get_winid
import prediction_commons
import numpy as np
import re
from pyliftover import LiftOver
from features_preprocess import WGBS_preprocess

In [2]:
def wgbs_sites_selection(tss,allsites):
    tss = tss.sort_values(['chr','coordinate'])
    allsites = all_sites.sort_values(['chr','coordinate'])
    i = 0
    selected_sites = []
    #selected_sites = pd.DataFrame(columns=['chr','coordinate','tss_coordinate'])
    tss['before'] = tss['coordinate']-100000
    tss['after'] = tss['coordinate']+100000
    for row in allsites.iterrows():
        if i >= len(tss):
            break
        chr = row[1]['chr']
        coordinate = row[1]['coordinate']
        winid = row[1]['winid']
        if chr==tss.ix[i,'chr'] and coordinate>=tss.ix[i,'before'] and coordinate<=tss.ix[i,'after']:
            selected_sites.extend([[winid,chr,coordinate,tss.ix[i,'chr'],tss.ix[i,'coordinate']]])
        else:
            while  i<len(tss) and (chr>tss.ix[i,'chr'] or (chr==tss.ix[i,'chr'] and coordinate>tss.ix[i,'after'])):
                i += 1
            if i<len(tss) and chr==tss.ix[i,'chr'] and coordinate>=tss.ix[i,'before'] and coordinate<=tss.ix[i,'after']:
                selected_sites.extend([[winid,chr,coordinate,tss.ix[i,'chr'],tss.ix[i,'coordinate']]])
    df = pd.DataFrame(selected_sites,columns=['winid','chr','coordinate','tss_chr','tss_coordinate'])
    df['chr'] = df['chr'].astype('i8')
    return df


def nearest_tss(tss,sites_df):
    merged = pd.merge(sites_df,tss,how='outer',on=['chr','coordinate'])
    merged.sort_values(['chr','coordinate'],inplace=True)
    merged.rename(columns={'strand':'before_tss'},inplace=True)
    merged.ix[merged['before_tss'].isnull()==False, 'before_tss'] = merged.ix[merged['before_tss'].isnull()==False,'coordinate']
    merged['after_tss'] = merged['before_tss']
    merged['before_tss'].fillna(method='ffill', inplace=True)
    merged['after_tss'].fillna(method='bfill',inplace=True)
    merged['dist_to_before_tss'] = np.abs(merged['coordinate']-merged['before_tss'])
    merged['dist_to_after_tss'] = np.abs(merged['coordinate']-merged['after_tss'])
    merged['tss'] = None
    before_ix = (merged['dist_to_before_tss'] < merged['dist_to_after_tss']) | (merged['dist_to_after_tss'].isnull())
    merged.ix[before_ix,'tss'] = merged.ix[before_ix,'before_tss']
    after_ix = (merged['dist_to_before_tss'] >= merged['dist_to_after_tss']) | (merged['dist_to_before_tss'].isnull())
    merged.ix[after_ix,'tss'] = merged.ix[after_ix,'after_tss']
    merged['dist_to_nearest_tss'] = np.abs(merged['coordinate']-merged['tss'])
    merged.drop(['before_tss','after_tss','tss','dist_to_before_tss','dist_to_after_tss'],axis=1,inplace=True)
    merged.dropna(axis=0,inplace=True)
    return merged

def rename_features(x):   #rename repetitive features
    features = np.array(x.columns)
    features_count = pd.Series(index=x.columns.unique())
    features_count = features_count.fillna(int(0))
    for i,name in enumerate(x.columns):
        if features_count[name] == 0:
            features_count[name] += 1
        else:
            features[i] = name+str(features_count[name])
            features_count[name] += 1
    x.columns = features
    return 

def read_WGBS(file):
    bed = pd.read_csv(file,usecols=[0,1,2,5,9,10],header=None,names=['chr','pos1','pos2','strand','total','percent'],sep='\s+')
    bed['coordinate'] = np.where(bed['strand']=='+',bed['pos1'],bed['pos1']-1)
    bed.drop(['pos1','pos2'],axis=1,inplace=True)
    bed['count'] = np.round(bed['total']*bed['percent']/100.0)
    bed.drop(['total','percent'],axis=1,inplace=True)
    bed = bed.groupby(['chr','coordinate']).aggregate({'count':sum}).reset_index()
    
    #    bed_counts = bed.groupby(['chr','coordinate']).aggregate({'count':sum})
    return bed

def hg38tohg19(row):
    global lo
    hg19 = lo.convert_coordinate('chr'+str(row[1]['chr']),row[1]['start'])
    if(len(hg19)>0):
        Chr,start,strand,score = hg19[0]
        try:
            Chr = int(Chr[3:])
        except ValueError:
            if Chr[3:] == 'X':
                Chr = 23
            elif Chr[3:] == 'Y':
                Chr = 24
        if strand == '+':
            start = start+1
        end = start+1
        return [Chr,start,end,row[1]['chr'],row[1]['start']]
    else:
        return [None,None,None,row[1]['chr'],row[1]['start']]

In [3]:
dataset = 'WGBS'
win_path= home+'data/wins.txt'
chrs=np.arange(1,22,dtype='int64')
wins = get_winid.read_wins(win_path,chrs)
all_wgbs_sites_file = home+'data/'+dataset+'/all_wgbs_sites_winid.csv'
hg19_wgbs_file = home+'data/'+dataset+'/hg19_WGBS.csv'

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21']


In [57]:
###get all WGBS sites only need to run once
data_dir = extra_storage+'WGBS/'
file = data_dir+'ENCFF844EFX.bed'
wgbs_file = home+'data/'+dataset+'/WGBS.bed'
bed = read_WGBS(file)
bed = get_winid.convert_chr_to_num(bed,chrs).sort_values(['chr','coordinate'])
bed.rename({'coordinate':'start'},axis=1,inplace=True)
bed['end'] = bed['start']+1
bed.drop(['count'],axis=1,inplace=True)
bed.to_csv(wgbs_file,columns=['chr','start','end'],index=False,sep="\t")

['1', '16', '15', '2', '19', '12', '5', '11', '21', '18', '4', '17', '14', '7', '3', '9', '20', '8', '10', '6', '13']


In [97]:
lo = LiftOver('hg19', 'hg38')
lo.convert_coordinate('chr1',10468)

[('chr1', 10468, '+', 20851231461)]

In [63]:
##convert to hg19, only need run once
lo = LiftOver('hg38', 'hg19')
coord_hg19 = [hg38tohg19(row)for row in bed.iterrows()]
coord_hg19 = pd.DataFrame(coord_hg19,columns=['chr','coordinate','end','hg38chr','hg38coordinate']).query('chr in @chrs')
coord_hg19.dropna().drop_duplicates(['chr','coordinate']).to_csv(hg19_wgbs_file,index=False)

In [10]:
from importlib import reload
reload(CADD_Preprocess)

<module 'features_preprocess.CADD_Preprocess' from '/home/ec2-user/git/EnsembleCpG/code/features_preprocess/CADD_Preprocess.py'>

In [66]:
#using WGBS(hg19) sites only run once
hg19_wgbs = pd.read_csv(hg19_wgbs_file,usecols=[0,1,3,4]).sort_values(['hg38chr','hg38coordinate']).reset_index(drop=True)
#hg19_wgbs = get_winid.convert_chr_to_num(hg19_wgbs,chrs)
all_sites = get_winid.get_winid(wins,hg19_wgbs,True).dropna()
all_sites['winid'] = all_sites['winid'].astype('i8')
all_sites.to_csv(all_wgbs_sites_file,index=False)

In [69]:
chrs = all_sites['chr'].unique()
cols=['chr', 'coordinate','strand']
tss =  pd.read_csv(home+'tss.txt',sep='\s+',header=None,names=cols,skiprows=1)
tss = get_winid.convert_chr_to_num(tss,chrs)

['1', '16', '15', '2', '19', '12', '5', '11', '21', '18', '4', '17', '14', '7', '3', '9', '20', '8', '10', '6', '13']


In [70]:
selected_wgbs_tss = wgbs_sites_selection(tss,all_sites)
with pd.HDFStore(home+'data/'+dataset+'/all_selected_wgbs_sites','w') as h5s:
    h5s['all_wgbs'] = selected_wgbs_tss

/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/ipykernel/__main__.py:15: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [4]:
with pd.HDFStore(home+'data/'+dataset+'/all_selected_wgbs_sites','r') as h5s:
    selected_wgbs_tss = h5s['all_wgbs'] 
###split into batchs of CpG around tss sites
start = prediction_commons.tss_start
end = prediction_commons.tss_end
selected_wgbs = selected_wgbs_tss[start:end]
sites_file = home+'data/'+dataset+'/all_sites_winid.csv'
selected_wgbs.to_csv(sites_file,index=False)
selected_wgbs.to_csv(home+'data/'+dataset+'/selected_pos_winid.csv',columns=['winid'],index=False,header=None)
additional_feature_file = home+'data/features/'+dataset+'/addtional_features_'+str(start)+'_'+str(end)

In [5]:
subprocess.call([home+'code/features_preprocess/Feature_export.R',dataset])

0

In [5]:
WGBS_h5s = home+'data/WGBS_single_H5S'
WGBS_proc = WGBS_preprocess.WGBS_Preprocess(h5s_file=WGBS_h5s,data_dir=extra_storage+'WGBS/',sites_file=sites_file,additional_feature_file=additional_feature_file,hg19_file= home+'data/WGBS/hg19_WGBS.csv')
if not os.path.exists(WGBS_h5s):
    WGBS_proc.process()
WGBS_proc.scores()

/ENCFF003JVR is done
/ENCFF043NUK is done
/ENCFF064GJQ is done
/ENCFF092FNE is done
/ENCFF103DNU is done
/ENCFF116DGM is done
/ENCFF121VIX is done
/ENCFF121ZES is done
/ENCFF164EAU is done
/ENCFF168HTX is done
/ENCFF179VKR is done
/ENCFF189WPY is done
/ENCFF200MJQ is done
/ENCFF210XTE is done
/ENCFF219GCQ is done
/ENCFF223LJW is done
/ENCFF247ILV is done
/ENCFF254DBF is done
/ENCFF266NGW is done
/ENCFF279HCL is done
/ENCFF297CJG is done
/ENCFF303ZGP is done
/ENCFF315ZJB is done
/ENCFF318AMC is done
/ENCFF331VRY is done
/ENCFF333OHK is done
/ENCFF355UVU is done
/ENCFF366UWF is done
/ENCFF428TVT is done
/ENCFF459EEM is done
/ENCFF477AUC is done
/ENCFF477GKI is done
/ENCFF479QJK is done
/ENCFF487XOB is done
/ENCFF489CEV is done
/ENCFF500DKA is done
/ENCFF510EMT is done
/ENCFF511FUP is done
/ENCFF513ITC is done
/ENCFF536RSX is done
/ENCFF545MIY is done
/ENCFF550FZT is done
/ENCFF553HJV is done
/ENCFF560SMW is done
/ENCFF575GIN is done
/ENCFF601NBW is done
/ENCFF623RHB is done
/ENCFF625GVK 

In [6]:
ATAC_h5s = home+'data/ATAC_H5S'
if os.path.exists(ATAC_h5s):
    atac_process = BED_Preprocess.BED_Preprocessing(h5s_file=ATAC_h5s,sites_file=sites_file,additional_feature_file=additional_feature_file,data_type='ATAC')
    atac_process.process()
else:
    atac_binning = BED_binning.BED_binning(data_type='ATAC',data_dir=extra_storage+'ATAC/',output=ATAC_h5s,sorted=True)
    atac_binning.binning()
    atac_process = BED_Preprocess.BED_Preprocessing(h5s_file=ATAC_h5s,sites_file=sites_file,additional_feature_file=additional_feature_file,data_type='ATAC')
    atac_process.process()  

/ENCFF048IOT is done
/ENCFF049DWX is done
/ENCFF069SZE is done
/ENCFF086TID is done
/ENCFF105ASY is done
/ENCFF111ULL is done
/ENCFF114WRE is done
/ENCFF135RTS is done
/ENCFF159RKV is done
/ENCFF168OTV is done
/ENCFF205KDV is done
/ENCFF217OUE is done
/ENCFF277DNH is done
/ENCFF293BRN is done
/ENCFF294ZCT is done
/ENCFF305PRP is done
/ENCFF310VHO is done
/ENCFF325VEG is done
/ENCFF372HEU is done
/ENCFF374OSU is done
/ENCFF374VWZ is done
/ENCFF377DAO is done
/ENCFF388DNI is done
/ENCFF429JLL is done
/ENCFF436NOT is done
/ENCFF468QNB is done
/ENCFF469OER is done
/ENCFF478ARL is done
/ENCFF479UTZ is done
/ENCFF482CAN is done
/ENCFF482HAC is done
/ENCFF585YAB is done
/ENCFF588PIS is done
/ENCFF592ALR is done
/ENCFF607XLB is done
/ENCFF608BWH is done
/ENCFF628MCV is done
/ENCFF629VEK is done
/ENCFF654ZNI is done
/ENCFF656OYT is done
/ENCFF670GFY is done
/ENCFF675QOY is done
/ENCFF678AIF is done
/ENCFF682RLN is done
/ENCFF692DLP is done
/ENCFF710ELD is done
/ENCFF711SUI is done
/ENCFF733PNQ 

In [7]:
RNASeq_h5s = home+'data/RNASeq/'
if len(os.listdir(RNASeq_h5s))>0:
    rnaseq_process = BED_Preprocess.BED_Preprocessing(h5s_file=RNASeq_h5s,sites_file=sites_file,additional_feature_file=additional_feature_file, data_type='RNASeq')
    rnaseq_process.process()
else:
    subprocess.call(['python',home+'code/feature_preprocess/RNASeq_binning.py'])
    rnaseq_process = BED_Preprocess.BED_Preprocessing(h5s_file=RNASeq_h5s,sites_file=sites_file,additional_feature_file=additional_feature_file, data_type='RNASeq')
    rnaseq_process.process()

/ENCFF000HGE is done
/ENCFF016ARQ is done
/ENCFF984YOJ is done
/ENCFF934GLU is done
/ENCFF254TPI is done
/ENCFF749IFE is done
/ENCFF369DYD is done
/ENCFF280SGM is done
/ENCFF028DDF is done
/ENCFF054FEC is done
/ENCFF902VJF is done
/ENCFF000HGF is done
/ENCFF549UBP is done
/ENCFF997MIE is done
/ENCFF170YVW is done
/ENCFF422EVZ is done
/ENCFF226QAW is done
/ENCFF269JPQ is done
/ENCFF658SRY is done
/ENCFF523EQM is done
/ENCFF839OMU is done
/ENCFF436WXH is done
/ENCFF901GJD is done
/ENCFF370ZLG is done
/ENCFF097NUI is done
/ENCFF955THM is done
/ENCFF081MWH is done
/ENCFF822SCG is done
/ENCFF380FFO is done
/ENCFF303BKD is done
/ENCFF158LBE is done
/ENCFF232AJD is done
/ENCFF818RWN is done
/ENCFF677JVE is done
/ENCFF022AJB is done
/ENCFF455QWU is done
/ENCFF198UFI is done
/ENCFF723ZMR is done
/ENCFF301ROZ is done
/ENCFF888ZFS is done
/ENCFF105THO is done
/ENCFF760IDU is done
/ENCFF624VBI is done
/ENCFF552FTX is done
/ENCFF623UTC is done
/ENCFF535JQR is done


/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/ipykernel/__main__.py:4: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->floating,key->block1_values] [items->['winid']]



In [ ]:
cadd_preprocess = CADD_Preprocess.CADD_Preprocess(sites_file=sites_file,additional_feature_file=additional_feature_file)
cadd_preprocess.process()

['12', 4419538, -0.03719, -0.08456766666666667, 2.243, 1.89]
['12', 4500718, -0.710229, -0.7493590000000001, 0.068, 0.05766666666666667]
['12', 4619098, 0.034059, -0.01809633333333333, 2.921, 2.464]
['12', 4715960, 0.049136, -0.058553333333333346, 3.074, 2.151]
['12', 4830295, -0.054721, -0.14403233333333335, 2.091, 1.5279999999999998]
['12', 4922070, 0.232722, 0.15279800000000002, 5.02, 4.177]
['12', 5006792, -0.77934, -0.8339483333333333, 0.048, 0.03833333333333334]
['12', 5091922, -0.156636, -0.235986, 1.346, 0.9676666666666667]
['12', 5178337, -0.220489, -0.276253, 0.994, 0.789]
['12', 5486003, -0.005052, -0.10748800000000001, 2.538, 1.7539999999999998]
['12', 5576256, -0.448874, -0.5155113333333333, 0.294, 0.21766666666666667]
['12', 5666826, -0.051209, -0.10968866666666667, 2.121, 1.7269999999999996]
['12', 6241538, -0.704108, -0.767121, 0.07, 0.05333333333333334]
['12', 6290923, 0.088818, 0.033663, 3.489, 2.956666666666667]
['12', 6341768, 0.790584, 0.7257083333333334, 9.406, 8.

In [ ]:
dann_preprocess = DANN_Preprocess.DANN_Preprocess(sites_file=sites_file,additional_feature_file=additional_feature_file)
dann_preprocess.process()

In [ ]:
eigen_preprocess = Eigen_Preprocess.Eigen_Preprocess(sites_file=sites_file,additional_feature_file=additional_feature_file)
eigen_preprocess.process()

In [13]:
genocanyon_scores = extra_storage+'GenoCanyon/Results/'+dataset+'/selected_site_scores.txt'
data_dir=extra_storage+'GenoCanyon/Results/'+dataset+'/'
if os.path.exists(genocanyon_scores):
    genocanyon_preprocess = GenoCanyon_Preprocess.GenoCanyon_Preprocess(data_dir=data_dir,sites_file=sites_file,additional_feature_file=additional_feature_file)
    genocanyon_preprocess.process()
else:
    print('Run GenoCanyon R script first')

In [14]:
selected_wgbs = pd.read_csv(home+'data/'+dataset+'/all_sites_winid.csv')

In [15]:
feature_dir = home+'data/features/'+dataset+'/'
files = os.listdir(feature_dir)
pattern = '.*all.csv$'
reg = re.compile(pattern)
files = [name for name in files if len(reg.findall(name))>0]

In [16]:
for file in files:    
    feature = pd.read_csv(feature_dir+file)
    print(len(feature.columns))
    selected_wgbs = pd.concat([selected_wgbs,feature],axis=1)

31
267
317
73
80
735
303


In [17]:
rename_features(selected_wgbs)
selected_wgbs.shape

(1000000, 1811)

In [18]:
selected_wgbs.isnull().sum().max()

0

In [19]:
additional_features = ['ATAC','CADD','DANN','Eigen','GenoCanyon','RNASeq','WGBS']
#merge with additional features
with pd.HDFStore(additional_feature_file,'r') as h5s:
    for feature in additional_features:
        feature_frame = h5s[feature]
        selected_wgbs = pd.concat([selected_wgbs,feature_frame],axis=1)
selected_wgbs = selected_wgbs.loc[:,~selected_wgbs.columns.duplicated()]
selected_wgbs['chr'] = selected_wgbs['chr'].astype('i8')

In [20]:
#nearest tss distance    
chrs = selected_wgbs['chr'].unique()
cols=['chr', 'coordinate','strand']
tss =  pd.read_csv(home+'tss.txt',sep='\s+',header=None,names=cols,skiprows=1)
tss = get_winid.convert_chr_to_num(tss,chrs)
tss.sort_values(['chr','coordinate'],inplace=True)
selected_wgbs = nearest_tss(tss,selected_wgbs)


['10', '11', '12']


/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/ipykernel/__main__.py:31: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [21]:
selected_wgbs.shape

(1000000, 2074)

In [22]:
with pd.HDFStore(home+'data/'+dataset+'/all_features_'+str(start)+'_'+str(end),'w') as h5s:
    h5s['all_features'] = selected_wgbs